# Using dataclasses instead of functions


In [10]:
import dataclasses
import typing

In [11]:
import itertools

In [30]:
computation = itertools.islice(itertools.accumulate(itertools.count(10)), 10)
computation

In [31]:
computation?

Type:        islice
String form: <itertools.islice object at 0x10b899908>
Docstring:  
islice(iterable, stop) --> islice object
islice(iterable, start, stop[, step]) --> islice object

Return an iterator whose next() method returns selected values from an
iterable.  If start is specified, will skip all preceding elements;
otherwise, start defaults to zero.  Step defaults to one.  If
specified as another value, step determines how many values are 
skipped between successive calls.  Works like a slice() on a list
but returns an iterator.


We have no idea what this function will do till we execute it.

In [32]:
list(computation)

[10, 21, 33, 46, 60, 75, 91, 108, 126, 145]

In [36]:
@dataclasses.dataclass
class Count:
    start: int
    
    def __iter__(self):
        return self.gen()
    
    def gen(self):
        i = self.start
        while True:
            yield i
            i += 1


@dataclasses.dataclass
class Accumulate:
    iterable: typing.Iterable
    
    def __iter__(self):
        return self.gen()
    
    def gen(self):
        s = 0
        for x in self.iterable:
            s += x
            yield s

@dataclasses.dataclass
class ISlice:
    iterable: typing.Iterable
    stop: int
    
    def __iter__(self):
        return self.gen()
    
    def gen(self):
        n = 0
        for x in self.iterable:
            yield x
            n += 1
            if n == self.stop:
                return
            
            

In [37]:
new_computation = ISlice(Accumulate(Count(10)), 10)

In [38]:
list(new_computation)

[10, 21, 33, 46, 60, 75, 91, 108, 126, 145]

In [39]:
new_computation

ISlice(iterable=Accumulate(iterable=Count(start=10)), stop=10)

Can also do this for functions:

In [48]:
@dataclasses.dataclass
class Map:
    fn: typing.Callable
    
    def __call__(self, xs):
        return [self.fn(x) for x in xs]

@dataclasses.dataclass
class Filter:
    fn: typing.Callable
    
    def __call__(self, xs):
        return [x for x in xs if self.fn(x)]

@dataclasses.dataclass
class Compose:
    a: typing.Callable
    b: typing.Callable
    
    def __call__(self, xs):
        return self.a(self.b(xs))

In [53]:
def is_even(i):
    return i % 2 == 0

def add_two(i):
    return i + 2

f = Compose(Filter(is_even), Map(add_two))
f

Compose(a=Filter(fn=<function is_even at 0x10bbf2f28>), b=Map(fn=<function add_two at 0x10b84b158>))

In [54]:
f([1, 2, 3, 4, 5, 6, ])

[4, 6, 8]

Now we can "see inside" this computation. Why is this useful?

We could later optimize this series of operations, by traversing the datastructrues.

Conclusion:
* Python can do FP fine.
* If you treat function composition as data, then your computation becomes less opaque, but it can be just the same for the user.
* Python gives you some tools to hide what your object does from what it is. Make use of these to give your users more bang for their buck.
* Much better for debugging.